## Importing the Libraries

In [40]:
import numpy as np
import cv2
import sys

##  Reading points from text file

In [41]:
def readPoints(path):
    points = []
    with open(path) as file:
        for line in file:
            x,y = line.split()
            points.append((int(x),int(y)))
    return points

## Applying affine transform

In [42]:
def applyAffineTransform(src,srcTri,dstTri,size):
    warpMat = cv2.getAffineTransform(np.float32(srcTri),np.float32(dstTri))
    dst = cv2.warpAffine(src,warpMat,(size[0],size[1]),None,flags = cv2.INTER_LINEAR,borderMode = cv2.BORDER_REFLECT_101)
    return dst

## Blending triangular regions from img1 and img2 to img

In [43]:
def morphTriangle(img1,img2,img,t1,t2,t,alpha):
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))
    t1Rect = []
    t2Rect = []
    tRect = []
    
    for i in range(0,3):
        tRect.append(((t[i][0]-r[0]),(t[i][1]-r[1])))
        t1Rect.append(((t1[i][0]-r1[0]),(t1[i][1]-r1[1])))
        t2Rect.append(((t2[i][0]-r2[0]),(t2[i][1]-r2[1])))
        
    mask = np.zeros((r[3],r[2],3),dtype = np.float32)
    cv2.fillConvexPoly(mask,np.int32(tRect),(1.0,1.0,1.0),16,0)
    
    img1Rect = img1[r1[1]:r1[1]+r1[3],r1[0]:r1[0]+r1[2]]
    img2Rect = img2[r2[1]:r2[1]+r2[3],r2[0]:r2[0]+r2[2]]
    
    size = (r[2],r[3])
    warpImage1 = applyAffineTransform(img1Rect,t1Rect,tRect,size)
    warpImage2 = applyAffineTransform(img2Rect,t2Rect,tRect,size)
    
    imgRect = (1.0-alpha)*warpImage1+alpha*warpImage2
    
    img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]*(1-mask)+imgRect*mask
    

In [44]:
if __name__ == '__main__':
    filename1 = 'donald_trump.jpg'
    filename2 = 'hillary_clinton.jpg'
    alpha = 0.5
    
    img1 = cv2.imread(filename1)
    img2 = cv2.imread(filename2)
    
    img1 = np.float32(img1)
    img2 = np.float32(img2)
    
    points1 = readPoints(filename1+'.txt')
    points2 = readPoints(filename2+'.txt')
    points = []
    
    for i in range(0,len(points1)):
        x = (1-alpha)*points1[i][0]+alpha*points2[i][0]
        y = (1-alpha)*points1[i][1]+alpha*points2[i][1]
        points.append((x,y))
        
    imgMorph = np.zeros(img1.shape,dtype = img1.dtype)
    
    with open("tri.txt") as file:
        for line in file:
            x,y,z = line.split()
            
            x = int(x)
            y = int(y)
            z = int(z)
            
            t1 = [points1[x],points1[y],points1[z]]
            t2 = [points2[x],points2[y],points2[z]]
            t = [points[x],points[y],points[z]]
            
            morphTriangle(img1,img2,imgMorph,t1,t2,t,alpha)
        
    cv2.imshow("Morphed Face",np.uint8(imgMorph))
    cv2.waitKey(0)
            